<a href="https://colab.research.google.com/github/moveformyfuture/House_Price_Expectation/blob/main/House_Price_Expectation_ETR%2BRF%2BGBR%2BXGB%2BHGB%2BNGB%2BCAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import pandas as pd
from pandas import read_csv, set_option

from sklearn.model_selection import cross_val_score, train_test_split, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils import resample, shuffle

# 구글 드라이브에 접근할 수 있도록 아래 코드 입력
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# 데이터셋 설정
x_train=read_csv('/content/drive/MyDrive/Colab Notebooks/집값예측 프로젝트/train.csv')
x_test=read_csv('/content/drive/MyDrive/Colab Notebooks/집값예측 프로젝트/test.csv')

In [31]:
# 데이터셋 확인
x_train.describe()

,id,Overall Qual,Gr Liv Area,Garage Cars,Garage Area,Total Bsmt SF,1st Flr SF,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
count,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000
mean,675.500000,6.208889,1513.542222,1.870370,502.014815,1082.644444,1167.474074,1.560741,1972.987407,1985.099259,1978.471852,186406.312593
std,389.855743,1.338015,487.523239,0.652483,191.389956,384.067713,375.061407,0.551646,29.307257,20.153244,25.377278,78435.424758
min,1.000000,2.000000,480.000000,1.000000,100.000000,105.000000,480.000000,0.000000,1880.000000,1950.000000,1900.000000,12789.000000
25%,338.250000,5.000000,1144.000000,1.000000,368.000000,816.000000,886.250000,1.000000,1955.000000,1968.000000,1961.000000,135000.000000
50%,675.500000,6.000000,1445.500000,2.000000,484.000000,1009.000000,1092.500000,2.000000,1976.000000,1993.000000,1978.500000,165375.000000
75%,1012.750000,7.000000,1774.500000,2.000000,588.000000,1309.500000,1396.500000,2.000000,2002.000000,2004.000000,2002.000000,217875.000000
max,1350.000000,10.000000,4476.000000,5.000000,1488.000000,2660.000000,2898.000000,4.000000,2010.000000,2010.000000,2207.000000,745000.000000


In [32]:
x_train[x_train['Garage Yr Blt']>=2022]

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
254,255,8,1564,Gd,2,502,Ex,1546,1564,Gd,2,2006,2007,2207,267300


In [33]:
x_train[x_train['Year Built']==2006]

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
19,20,6,1326,Gd,2,427,Gd,1326,1326,Gd,2,2006,2006,2006,174190
67,68,8,1460,Gd,2,480,Gd,1460,1460,Ex,2,2006,2007,2006,247000
97,98,6,1495,Gd,2,440,Gd,738,738,Gd,2,2006,2006,2006,171925
102,103,8,1324,Gd,2,550,Gd,1324,1324,Gd,2,2006,2006,2006,254000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288,1289,7,1977,Gd,3,574,Gd,982,982,Gd,2,2006,2006,2006,250000
1327,1328,8,1634,Gd,3,866,Gd,1626,1634,Gd,2,2006,2006,2006,250000
1328,1329,8,1370,Gd,2,484,Gd,1370,1370,Gd,2,2006,2006,2006,205000
1336,1337,10,2698,Ex,3,736,Ex,1850,1850,Ex,2,2006,2006,2006,535000


In [ ]:
# train 데이터 수정 (2207 비정상적 숫자를 2006으로 변경)
x_train.loc[(x_train['Garage Yr Blt'] == 2207), 'Garage Yr Blt']=2006

In [36]:
# 범주 형식의 데이터를 숫자로 변환(Label-인코딩 방법)
x_train=x_train.copy()
x_train[x_train.columns[x_train.dtypes=='O']] = x_train[x_train.columns[x_train.dtypes=='O']].astype(str).apply(LabelEncoder().fit_transform)
# astype() : 데이터 타입 바꾸기, LabelEncoder() : 원 핫 인코딩
# 데이터 타입이 object인 columns만 선택해서 indexing = str로 데이터 타입을 바꾼 후 LabelEncoder() 함수를 적용
x_train.info() # 데이터 변경 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              1350 non-null   int64
 1   Overall Qual    1350 non-null   int64
 2   Gr Liv Area     1350 non-null   int64
 3   Exter Qual      1350 non-null   int64
 4   Garage Cars     1350 non-null   int64
 5   Garage Area     1350 non-null   int64
 6   Kitchen Qual    1350 non-null   int64
 7   Total Bsmt SF   1350 non-null   int64
 8   1st Flr SF      1350 non-null   int64
 9   Bsmt Qual       1350 non-null   int64
 10  Full Bath       1350 non-null   int64
 11  Year Built      1350 non-null   int64
 12  Year Remod/Add  1350 non-null   int64
 13  Garage Yr Blt   1350 non-null   int64
 14  target          1350 non-null   int64
dtypes: int64(15)
memory usage: 158.3 KB


In [35]:
# 변수 선택을 위해 후진제거법 사용